# Lorenz 96 Ginelli Workbook

This notebook is a space for quick investigations & current implementation notes. Less formal.

In [1]:
# Imports

import numpy as np
import time as tm
import pickle
import os
import xarray as xr
import sys
import ipywidgets as widgets
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

In [2]:
# Progress Bar Import

def in_notebook():
    """
    Returns ``True`` if the module is running in IPython kernel,
    ``False`` if in IPython shell or other Python shell.
    """
    return 'ipykernel' in sys.modules

if in_notebook():
    from tqdm.notebook import tqdm #tqdm is progress bar
else:
    from tqdm import tqdm

### Notes

- Need to check how 'lyapobs' are saving when it goes beyond 10 dumps.
- Problem with last FTBLE

In [ ]:
def spectra (data, geometry = 'C'):
    if (geometry == 'C'):
        spectra = data.ftcle.mean(dim='time', skipna=True)
    else: # then FTBLE
        spectra = data.ftble.mean(dim='time', skipna=True)
    return spectra

In [ ]:
def spectra_parameter(data_list, parameter_key, geometry='C', save = 'None'):
    """Plots Lyapunov spectra for a varying parameter.
    data_list, list of data arrays containing lyapunov observations
    parameter_key, the parameter from data.attrs that we're investigating.
    geometry, CLEs or BLEs.
    save, file save name
    """

    for data in data_list:
        
        attr = data.attrs[parameter_key]
       
        if (geometry == 'C'):
            spectra = data.ftcle.mean(dim='time', skipna=True)
            label = f'{parameter_key}$={attr:.2f}$'
        else: # then FTBLE
            spectra = data.ftble.mean(dim='time', skipna=True)
            label = f'{parameter_key}$={attr:.2f}$'


        plt.plot(spectra, '-' ,label=label)

    plt.legend()
    plt.title(f'FT{geometry}LE Means')
    plt.xlabel('LE Index') 
    
    if (save == 'None'):
        plt.show()
    else:
        print('Saving')
        plt.savefig(save, dpi=1200)
        plt.show()
    

## Checking $k_a$

In [3]:
def fetch_observations(directory):
    observations = [xr.open_dataset(f'{directory}/step5/{file}') for file in os.listdir(f'{directory}/step5')]
    lyap_data = xr.concat(observations, 'time')
    sort = lyap_data.sortby('time')
    sort.to_netcdf(f'{directory}/step5/Processed.nc')
    print(f'Processed data written to {directory}/step5/Processed.nc\n')
    print('Closing up')
    for ds in observations:
        ds.close()
    return sort

def fetch_experiment(directory):
    data = []
    for i in tqdm([int(s) for s in os.listdir(f'{directory}')]):
        print(f'Fetching {directory}/{i}')
        ds = fetch_observations(f'{directory}/{i}')
        data.append(ds)
    return data

In [4]:
directory = 'h-effect'
for i in tqdm([1, 2]):
    print(f'Fetching {directory}/{i}')
    fetch_observations(f'{directory}/{i}')

Fetching h-effect/1
Processed data written to h-effect/1/step5/Processed.nc

Closing up
Fetching h-effect/2
Processed data written to h-effect/2/step5/Processed.nc

Closing up



In [ ]:
for data in kb_data:
    plt.figure()
    plt.plot(data.time)
    plt.show()